In [1]:
#The sudo apt-get update command is used to download package information from all configured sources
!sudo apt update

# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar -xvf spark-3.3.0-bin-hadoop3.tgz

# install findspark using pip
!pip install -q findspark

#For python users, also install pyspark using the following command.
!pip install pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  

In [2]:
# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [3]:
#Run a local spark session to test your installation:

import findspark
findspark.init()

#to know the location where Spark is installed
findspark.find()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from google.colab import files
files.upload()

In [5]:
!unzip '/content/Data.zip'

Archive:  /content/Data.zip
   creating: Data/
  inflating: Data/location_temp.csv  
  inflating: Data/server_name.csv    
  inflating: Data/utilization.csv    
  inflating: Data/utlization.json    


In [6]:
data_path = '/content/Data/'

In [7]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession \
    .builder \
    .appName("Spark SQL Query Dataframes") \
    .getOrCreate()

In [31]:
file_path = data_path + "/utilization.csv"
df1 = spark.read.format("csv").option("header", "false").load(file_path)

In [32]:
df1.head()

Row(_c0='03/05/2019 08:06:14', _c1='100', _c2='0.57', _c3='0.51', _c4='47')

In [33]:
df1.columns

['_c0', '_c1', '_c2', '_c3', '_c4']

In [35]:
df2 = df1.withColumnRenamed("_c0", "event_datetime") \
        .withColumnRenamed ("_c1", "server_id")       \
        .withColumnRenamed("_c2", "cpu_utilization")  \
        .withColumnRenamed("_c3", "free_memory")      \
        .withColumnRenamed("_c4", "session_count")

In [40]:
json_df2_path = data_path + "/utilization.json"
df2.write.mode('overwrite').json(json_df2_path)

In [41]:
json_df2_path = data_path + "/utilization.json"
df = spark.read.format("json").load(json_df2_path)

In [42]:
df.show(5)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
+---------------+-------------------+-----------+---------+-------------+
only showing top 5 rows



In [43]:
df.printSchema()

root
 |-- cpu_utilization: string (nullable = true)
 |-- event_datetime: string (nullable = true)
 |-- free_memory: string (nullable = true)
 |-- server_id: string (nullable = true)
 |-- session_count: string (nullable = true)



In [44]:
df.count()

500000

In [45]:
df.createOrReplaceTempView("utilization")

In [46]:
df_sql = spark.sql("SELECT * FROM utilization LIMIT 10")
df_sql.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|       0.40|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|       0.60|      100|           45|
+---------------+-------------------+-

In [47]:
df_sql.count()

10

In [48]:
df_sql = spark.sql("SELECT server_id, session_count FROM utilization LIMIT 10")
df_sql.show()

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      100|           47|
|      100|           43|
|      100|           62|
|      100|           50|
|      100|           43|
|      100|           48|
|      100|           58|
|      100|           58|
|      100|           62|
|      100|           45|
+---------+-------------+



In [50]:
df_sql = spark.sql("SELECT server_id as sid, session_count as sc FROM utilization")
df_sql.show(5)

+---+---+
|sid| sc|
+---+---+
|100| 47|
|100| 43|
|100| 62|
|100| 50|
|100| 43|
+---+---+
only showing top 5 rows



In [51]:
df_sql = spark.sql("SELECT * FROM utilization WHERE server_id = 120")
df_sql.show(10)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.66|03/05/2019 08:06:48|       0.31|      120|           54|
|           0.58|03/05/2019 08:11:48|       0.38|      120|           64|
|           0.55|03/05/2019 08:16:48|       0.61|      120|           54|
|           0.70|03/05/2019 08:21:48|       0.35|      120|           80|
|           0.60|03/05/2019 08:26:48|       0.39|      120|           71|
|           0.53|03/05/2019 08:31:48|       0.35|      120|           49|
|           0.73|03/05/2019 08:36:48|       0.42|      120|           73|
|           0.41|03/05/2019 08:41:48|       0.60|      120|           72|
|           0.62|03/05/2019 08:46:48|       0.57|      120|           57|
|           0.67|03/05/2019 08:51:48|       0.44|      120|           78|
+---------------+-------------------+-

In [52]:
df_sql.count()

10000

In [53]:
df_sql = spark.sql("SELECT server_id, session_count FROM utilization WHERE session_count > 70")
df_sql.show(10)

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      100|           71|
|      100|           71|
|      100|           71|
|      100|           71|
|      100|           72|
|      100|           72|
|      100|           71|
|      100|           71|
|      100|           71|
|      100|           72|
+---------+-------------+
only showing top 10 rows



In [54]:
df_sql.count()

239659

In [55]:
df_sql = spark.sql("SELECT server_id, session_count FROM utilization WHERE session_count > 70 AND server_id = 120")
df_sql.show(10)

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      120|           80|
|      120|           71|
|      120|           73|
|      120|           72|
|      120|           78|
|      120|           73|
|      120|           78|
|      120|           73|
|      120|           74|
|      120|           78|
+---------+-------------+
only showing top 10 rows



In [56]:
df_sql.count()

2733

In [57]:
df_sql = spark.sql("SELECT server_id, session_count \
                   FROM utilization \
                   WHERE session_count > 70 AND server_id = 120 \
                   ORDER BY session_count DESC")
df_sql.show(10)

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
+---------+-------------+
only showing top 10 rows



In [59]:
df_sql = spark.sql("SELECT distinct server_id, session_count \
                   FROM utilization \
                   WHERE session_count > 70 \
                   ORDER BY session_count DESC")
df_sql.show(10)

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      103|           99|
|      102|           99|
|      101|           99|
|      108|           99|
|      149|           99|
|      145|           99|
|      148|           99|
|      137|           99|
|      133|           99|
|      113|           99|
+---------+-------------+
only showing top 10 rows



In [60]:
df_sql = spark.sql("SELECT distinct server_id, session_count \
                   FROM utilization \
                   WHERE session_count > 70 \
                   ORDER BY server_id DESC")
df_sql.show(10)

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      149|           95|
|      149|           97|
|      149|           81|
|      149|           86|
|      149|           77|
|      149|           82|
|      149|           85|
|      149|           88|
|      149|           87|
|      149|           90|
+---------+-------------+
only showing top 10 rows



In [61]:
df_sql = spark.sql("SELECT count(*) \
                    FROM utilization \
                    WHERE session_count > 70")
df_sql.show()

+--------+
|count(1)|
+--------+
|  239659|
+--------+



In [63]:
df_sql = spark.sql("SELECT server_id, count(*) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id")
df_sql.show(10)

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      125|    2843|
|      101|    9808|
|      112|    7425|
|      113|    9418|
|      107|    5646|
|      110|    2826|
|      100|     391|
|      126|    6365|
|      120|    2733|
|      118|    7913|
+---------+--------+
only showing top 10 rows



In [66]:
df_sql = spark.sql("SELECT server_id, count(*) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show(10)

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      101|    9808|
|      113|    9418|
|      145|    9304|
|      103|    8744|
|      102|    8586|
|      133|    8583|
|      108|    8375|
|      149|    8288|
|      137|    8248|
|      148|    8027|
+---------+--------+
only showing top 10 rows



In [67]:
df_sql = spark.sql("SELECT server_id, min(session_count), avg(session_count), max(session_count) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show(10)

+---------+------------------+------------------+------------------+
|server_id|min(session_count)|avg(session_count)|max(session_count)|
+---------+------------------+------------------+------------------+
|      101|               100| 87.66557911908646|                99|
|      113|               100| 86.96262476109577|                99|
|      145|               100| 86.97732158211522|                99|
|      103|               100| 85.76372369624886|                99|
|      102|               100| 85.71150710458886|                99|
|      133|               100| 85.46720260981009|                99|
|      108|               100|  85.1219104477612|                99|
|      149|                71|  84.9612693050193|                99|
|      137|                71|  85.0061833171678|                99|
|      148|                71| 84.70350068518749|                99|
+---------+------------------+------------------+------------------+
only showing top 10 rows



In [68]:
df_sql = spark.sql("SELECT server_id, min(session_count), round(avg(session_count),2), max(session_count) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show(10)

+---------+------------------+----------------------------+------------------+
|server_id|min(session_count)|round(avg(session_count), 2)|max(session_count)|
+---------+------------------+----------------------------+------------------+
|      101|               100|                       87.67|                99|
|      113|               100|                       86.96|                99|
|      145|               100|                       86.98|                99|
|      103|               100|                       85.76|                99|
|      102|               100|                       85.71|                99|
|      133|               100|                       85.47|                99|
|      108|               100|                       85.12|                99|
|      149|                71|                       84.96|                99|
|      137|                71|                       85.01|                99|
|      148|                71|                      

In [69]:
csv_df_path = data_path + "/server_name.csv"
df_server = spark.read.csv(csv_df_path,header=True)

In [70]:
df_server.show(10)

+---------+-----------+
|server_id|server_name|
+---------+-----------+
|      100| 100 Server|
|      101| 101 Server|
|      102| 102 Server|
|      103| 103 Server|
|      104| 104 Server|
|      105| 105 Server|
|      106| 106 Server|
|      107| 107 Server|
|      108| 108 Server|
|      109| 109 Server|
+---------+-----------+
only showing top 10 rows



In [71]:
df_server.createOrReplaceTempView("server_name")

In [72]:
df_count = spark.sql("SELECT DISTINCT server_id FROM utilization ORDER BY server_id")
df_count.show(10)

+---------+
|server_id|
+---------+
|      100|
|      101|
|      102|
|      103|
|      104|
|      105|
|      106|
|      107|
|      108|
|      109|
+---------+
only showing top 10 rows



In [73]:
spark.sql("SELECT min(server_id), max(server_id) FROM utilization").show()

+--------------+--------------+
|min(server_id)|max(server_id)|
+--------------+--------------+
|           100|           149|
+--------------+--------------+



In [76]:
spark.sql("SELECT * FROM server_name").show(5)

+---------+-----------+
|server_id|server_name|
+---------+-----------+
|      100| 100 Server|
|      101| 101 Server|
|      102| 102 Server|
|      103| 103 Server|
|      104| 104 Server|
+---------+-----------+
only showing top 5 rows



In [77]:
df_join = spark.sql("SELECT u.server_id, sn.server_name, u.session_count \
                     FROM utilization u \
                     INNER JOIN server_name sn \
                     ON sn.server_id = u.server_id")
df_join.show(5)  

+---------+-----------+-------------+
|server_id|server_name|session_count|
+---------+-----------+-------------+
|      100| 100 Server|           47|
|      100| 100 Server|           43|
|      100| 100 Server|           62|
|      100| 100 Server|           50|
|      100| 100 Server|           43|
+---------+-----------+-------------+
only showing top 5 rows



In [78]:
from pyspark.sql import Row

In [83]:
df_dup = spark.sparkContext.parallelize([Row(server_name='101 Server', cpu_utilization=85, session_count=80), \
                             Row(server_name='101 Server', cpu_utilization=80, session_count=90),
                             Row(server_name='102 Server', cpu_utilization=85, session_count=80),
                             Row(server_name='102 Server', cpu_utilization=85, session_count=80),
                             Row(server_name='103 Server', cpu_utilization=87, session_count=82),
                             Row(server_name='103 Server', cpu_utilization=87, session_count=82),
                             Row(server_name='104 Server', cpu_utilization=88, session_count=81),
                             Row(server_name='104 Server', cpu_utilization=88, session_count=81)]).toDF()

In [84]:
df_dup.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
| 102 Server|             85|           80|
| 103 Server|             87|           82|
| 103 Server|             87|           82|
| 104 Server|             88|           81|
| 104 Server|             88|           81|
+-----------+---------------+-------------+



In [85]:
df_dup.drop_duplicates().show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
| 103 Server|             87|           82|
| 104 Server|             88|           81|
+-----------+---------------+-------------+



In [86]:
df_dup.drop_duplicates(['server_name']).show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 102 Server|             85|           80|
| 101 Server|             85|           80|
| 104 Server|             88|           81|
| 103 Server|             87|           82|
+-----------+---------------+-------------+



In [87]:
df = spark.sparkContext.parallelize([Row(server_name='101 Server', cpu_utilization=85, session_count=80), \
                             Row(server_name='101 Server', cpu_utilization=80, session_count=90),
                             Row(server_name='102 Server', cpu_utilization=85, session_count=40),
                             Row(server_name='103 Server', cpu_utilization=70, session_count=80),
                             Row(server_name='104 Server', cpu_utilization=60, session_count=80)]).toDF()

In [88]:
df.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           40|
| 103 Server|             70|           80|
| 104 Server|             60|           80|
+-----------+---------------+-------------+



In [90]:
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

In [91]:
df_na = df.withColumn('na_col', lit(None).cast(StringType()))

In [92]:
df.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           40|
| 103 Server|             70|           80|
| 104 Server|             60|           80|
+-----------+---------------+-------------+



In [93]:
df_na = df.withColumn('na_col', lit(None).cast(StringType()))

In [94]:
df_na.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+



In [95]:
df_na.fillna('A').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
+-----------+---------------+-------------+------+



In [96]:
df2 = df_na.fillna('A').union(df_na)

In [97]:
df2.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+



In [98]:
df2.na.drop().show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
+-----------+---------------+-------------+------+



In [99]:
df2.createOrReplaceTempView("na_table")

In [100]:
spark.sql("SELECT * FROM na_table").show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           40|     A|
| 103 Server|             70|           80|     A|
| 104 Server|             60|           80|     A|
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+

